# Gerar a Camada Silver a partir da Bronze

Neste notebook iremos processar os dados da tabela de vendas na camada Gold, gravando o resultado na camada Silver. 
Neste caso nossa transformação será tirar o C das invoices que são canceladas e gerar uma nova coluna, tipo, que terá os seguintes valores:
- __I__: Invoice
- __C__: Cancelamento

Ao final a coluna InvoiceNo sera ajustada sem o C e convertida para numérica.

## Carregar a Camada Bronze
Definir e registrar como temporárias as tabelas da camada bronze. Para facilitar a identificação das mesmo será adicionado o prefixo 
_bronze antes do nome da tabela:

__Ex.:__ vendas -> bronze_vendas

In [1]:
from load_lakehouse import *

## Gerar a tabela de Vendas
A tabela de vendas será transformada para incluir o campo _Type_ seguindo a regra definida acima; 
o campo _InvoiceNo_ será transformado em bigint (long), porém, antes de processar este campo precisa retirar o C do inicio das invoices que foram canceladas.

In [2]:
df = %sql \
select \
    cast(case when left(InvoiceNo,1) = 'C' then substring(InvoiceNo, 2) else InvoiceNo end as bigint) as InvoiceNo, \
    case when left(InvoiceNo,1) = 'C' then 'C'else 'I' end as Type, \
    StockCode, \
    Description, \
    Quantity, \
    InvoiceDate, \
    UnitPrice, \
    CustomerID, \
    Country \
from bronze_vendas

Segue como ficou o schema da tabela

In [3]:
df.printSchema()

root
 |-- InvoiceNo: long (nullable = true)
 |-- Type: string (nullable = false)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



### Limpar Silver existente
Antes de salvar as tabelas da camada Silver, estas serão removidas para economizar espaço em disco.

In [4]:
%%bash
if [ -d lakehouse/silver/vendas ]; then
    rm -Rf lakehouse/silver/vendas
fi

if [ -d lakehouse/silver/stock ]; then
    rm -Rf lakehouse/silver/stock
fi

if [ -d lakehouse/silver/country ]; then
    rm -Rf lakehouse/silver/country
fi

## Salvar tabela de vendas

In [5]:
%%time

df.write.format("delta").mode("overwrite").save("lakehouse/silver/vendas")

CPU times: user 4.37 ms, sys: 0 ns, total: 4.37 ms
Wall time: 7.38 s


## Gerar tabelas derivadas
Analisando a tabela de vendas é possível perceber que podemos extrair tabelas derivadas desta:
1. __Stock__: tabela com os produtos comercializados
2. __Country__: tabela de países que tiveram alguma invoice

### Tabela Stock

In [6]:
%%time

spark.sql("""
select distinct
    StockCode,
    Description
from bronze_vendas
order by description
""").write.format("delta").mode("overwrite").save("lakehouse/silver/stock")

CPU times: user 3.02 ms, sys: 0 ns, total: 3.02 ms
Wall time: 2.24 s


### Tabela de Countries

In [7]:
%%time

spark.sql("""
select distinct
    country
from bronze_vendas
order by country
""").write.format("delta").mode("overwrite").save("lakehouse/silver/country")

CPU times: user 0 ns, sys: 3.3 ms, total: 3.3 ms
Wall time: 1.53 s
